In [1]:
import json
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

/Users/amoghkulkarni/.local/share/virtualenvs/gadget-spec-bot-PKdu65VV/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/amoghkulkarni/.local/share/virtualenvs/gadget-spec-bot-PKdu65VV/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "company": {"type": "text"},
            "name": {"type": "text"},
            "specifications": {"type": "keyword"} ,
            "name_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
            "specification_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

In [3]:
es_client = Elasticsearch('http://localhost:9200')

In [10]:
response = es_client.indices.exists(index="mobile-abc")

In [14]:
'abc' if es_client.indices.exists(index="mobile-abc") else 'def'

'def'

In [7]:
index_name = "mobile-specifications"
es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'mobile-specifications'})

In [8]:
import pickle

In [9]:
with open('../search_build_and_eval/mobile_specs_vector_data.pkl', 'rb') as fp:
    mobile_specs_vector_data = pickle.load(fp)

In [10]:
with open('../data-extraction/mobile_specifications_data.json', 'r') as fp:
    mobile_specs_data = json.load(fp)

In [11]:
model = SentenceTransformer("all-mpnet-base-v2")

/Users/amoghkulkarni/.local/share/virtualenvs/gadget-spec-bot-PKdu65VV/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
def elastic_search_knn(vector, vector_field="name_vector"):
    """Return elastic search results for given vector."""
    knn_query = {
        "field": vector_field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000
    }

    search_query = {
        'knn': knn_query,
        '_source': ["name", "company", "specifications"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [13]:
def get_mobile_name_results(mobile_name_statement):
    """Return the ES results of the mobile phone asked by user."""
    input_vector = model.encode(mobile_name_statement)
    mobile_context = elastic_search_knn(input_vector, 'specification_vector')

    return mobile_context

In [14]:
def build_prompt(phone_specifications, mobile_context):
    """Build the context using search results and return the prompt."""
    prompt_template = """
You are mobile phone expert. Answer all the specifications of PHONE using the CONTEXT provided from SPECFICATIONS database.
The specifications in the CONTEXT is a json string. Convert the json string to readable format before printing out the answer.
Use only the specifications given in the context to print the specs of the device.
Do not number the specifications passed.
Print only the first result in a markdown format where subsections are indented in readable format.


PHONE: {phone}

CONTEXT:
{context}
""".strip()

    context = ""

    for each in mobile_context:
        context = context + f"\n\nname: {each['name']}\ncompany: {each['company']}\nspecifications: {each['specifications']}"

    prompt = prompt_template.format(phone=phone_specifications, context=context)
    return prompt

In [15]:
def llm(model, client, prompt):
    """Return LLM generation based on context."""
    if model.startswith('ollama'):
        response = ollama_client.chat.completions.create(
            model='phi3',
            messages=[{"role": "user", "content": prompt}]
        )
        answer = response.choices[0].message.content
        tokens = {
            'prompt_tokens': response.usage.prompt_tokens,
            'completion_tokens': response.usage.completion_tokens,
            'total_tokens': response.usage.total_tokens
        }
    else:
        messages = [{"role": "user", "content": prompt}]
        response = client.chat_completion(messages, max_tokens=1000)
        
    return response.choices[0].message.content

In [16]:
def rag(model, client, mobile_phone_statement):
    """Run the RAG flow for the given mobile phone statement."""
    mobile_name_context = get_mobile_name_results(mobile_phone_statement)
    prompt = build_prompt(mobile_phone_statement, mobile_name_context)
    answer = llm(model, client, prompt)

    return answer

In [17]:
mobile_phone_statement = 'Print the list of all iphones present.'

In [21]:
import os
from huggingface_hub import InferenceClient

In [19]:
HF_API_TOKEN = os.getenv('HF_API_TOKEN')

In [22]:
hugging_face = InferenceClient("HuggingFaceH4/zephyr-7b-beta", token=HF_API_TOKEN)

In [23]:
result = rag('mistral', hugging_face, mobile_phone_statement)

In [31]:
mobile_name_context = get_mobile_name_results(mobile_phone_statement)

In [32]:
prompt = build_prompt(mobile_phone_statement, mobile_name_context)

In [33]:
messages = [{"role": "user", "content": prompt}]

In [35]:
response = hugging_face.chat_completion(messages, max_tokens=1000)

In [36]:
response.usage.prompt_tokens

148

In [24]:
print(result)

iPhones:

- iPhone 12:
  - Display: 6.1-inch Super Retina XDR OLED display with Ceramic Shield front cover, HDR display, True Tone, 2532-by-1170-pixel resolution at 460 ppi
  - Chip: A14 Bionic chip with 64-bit architecture, Neural Engine, 8-core graphics
  - Storage: 64GB, 128GB, or 256GB
  - Camera: Dual 12-megapixel Ultra Wide and Wide cameras with Night mode, Deep Fusion, Smart HDR 3, 4x optical zoom range, and 2x optical zoom in/out
  - Video Recording: 4K video recording at 24 fps, 30 fps, or 60 fps, 1080p video recording at 30 fps or 60 fps, 720p video recording at 30 fps
  - Battery: Up to 17 hours of talk time on a wireless network, up to 65 hours of audio playback, charges wirelessly via Qi charger or Lightning connector
- iPhone 12 mini:
  - Display: 5.4-inch Super Retina XDR OLED display with Ceramic Shield front cover, HDR display, True Tone, 2340-by-1080-pixel resolution at 476 ppi
  - Chip: A14 Bionic chip with 64-bit architecture, Neural Engine, 8-core graphics
  - Stor

In [25]:
mobile_phone_statement = 'Detailed specifications of iphone 16.'

In [26]:
result = rag('mistral', hugging_face, mobile_phone_statement)

In [27]:
print(result)

Display:
- IPHONE 16 features a 6.7-inch Super Retina XDR OLED display with a resolution of 2772 x 1284 pixels and a pixel density of 458 pixels per inch.
- The display has a peak brightness of 1200 nits, HDR support, and is protected by Ceramic Shield.

Processor:
- IPHONE 16 is powered by the A15 Bionic chip, a 64-bit CPU with six cores, and a four-core GPU.
- The chip is built on a 5nm process and provides improved performance and efficiency compared to the previous generation.

Storage:
- IPHONE 16 offers storage options of 128GB, 256GB, and 512GB.
- The storage is internal and cannot be expanded.

Rear Camera:
- The camera system on IPHONE 16 consists of a primary 12-megapixel ultra-wide camera and aprimary 12-megapixel wide-angle camera.
- The primary wide-angle camera has an aperture of f/1.8, support for Night mode, Deep Fusion, and Smart HDR 3.
- The ultra-wide camera has a 120-degree field of view and an aperture of f/2.4.

Front Camera:
- IPHONE 16 has a 12-megapixel front-f

In [28]:
mobile_phone_statement = 'List all the phones with 48 megapixel camera'

In [29]:
result = rag('mistral', hugging_face, mobile_phone_statement)

In [30]:
print(result)

PHONE with 48 Megapixel Camera:

In today's smartphone market, the camera quality is one of the major factors that decide a device's success. Several smartphone brands have introduced high-megapixel cameras, one of which is 48 megapixels. In this article, we will list down the smartphones that come with a 48-megapixel camera.

Camera:
- Primary Rear Camera: 48 Megapixels (f/1.8 aperture)

Display:
- Screen Size: 6.3 inches (1080 x 2340 pixels)
- Type: Super AMOLED, capacitive touchscreen, 16M colors, HDR10
- Protection: Corning Gorilla Glass (3rd generation), Scratch-resistant

Battery:
- Type: Li-Po 4000mAh, non-removable
- Fast charging 25W, 10W wireless charging, 4.5W reversible wireless charging 

Operating System:
- Android 10, MIUI 11

Processor & Storage:
- Chipset: Qualcomm SM6150 Snapdragon 675 (11 nm)
- CPU: Octa-core (2x2.0 GHz Kryo 460 & 6x1.7 GHz Kryo 460)
- GPU: Adreno 612
- RAM: 6GB/8GB
- Internal Memory: 64GB/128GB, expandable up to 512GB via microSD card (uses shared S

#### HuggingFaceH4/mistral-7b-sft-beta

In [44]:
mobile_phone_statement = 'Detailed specifications of iphone 16.'

In [47]:
hugging_face = InferenceClient("mistralai/Mistral-7B-Instruct-v0.2", token=HF_API_TOKEN)

In [48]:
result = rag('mistral', hugging_face, mobile_phone_statement)

In [50]:
from pprint import pprint